In [1]:
import re
import sys
import json
sys.path.append('./../')
sys.path.append('./../../')
import random
import string
import urllib, json
from math import *
from operator import * 
from itertools import *
from collections import *

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cosine
from scipy.spatial.distance import euclidean
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score

from sklearn.metrics import normalized_mutual_info_score, adjusted_mutual_info_score, adjusted_rand_score

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

from playgroundYelp import *
from modelYelp import *
session = getSession()

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

plotly.tools.set_credentials_file(username='josephcc', api_key='G56dPHPPVbSeiaU0KutN')

from terminaltables import AsciiTable

In [2]:
GS = {}
for term in ['cocktail', 'basil', 'lemon']:
    with open('../gs/%s.gs.json' % term) as jsonfile:
        GS[term] = json.load(jsonfile)

In [3]:
def getVectorsForToken(token, limit=10000, table='business_token_vector_pittsburgh'):
    vectors = session.execute('SELECT business_id, vector FROM %s WHERE city = :city AND token = :token LIMIT :limit' % table, {
        'token': token,
        'limit': limit,
        'city': 'Pittsburgh'
    })
    vectors = list(vectors)
    business_ids = map(itemgetter(0), vectors)
    vectors = map(itemgetter(1), vectors)
    return business_ids, vectors

In [4]:
def silhouetteAnalysis(vectors, bids, gs, algorithm=lambda n_clusters: KMeans(n_clusters=n_clusters), repeat=1, Ks=range(2,8)):
    print 'silhouetteAnalysis START'
    print '| - # of restaurant vectors: ', len(vectors)

    scores = []
    for n_clusters in Ks:
        bestScore = -2
        bestResults = None
        for _ in range(repeat):
            clusterer = algorithm(n_clusters)
            cluster_labels = clusterer.fit_predict(np.asarray(vectors))
            try:
                silhouette_avg = silhouette_score(vectors, cluster_labels, sample_size=100)
            except:
                continue
            if silhouette_avg >= bestScore:
                bestScore = silhouette_avg
                bestResults = cluster_labels
        scores.append((n_clusters, bestScore, bestScore * (n_clusters ** 0.5), bestResults))
    scores.sort(key=itemgetter(1), reverse=True)

    print '| Ranking of different Ks:'
    for n, score, weightedScore, results in scores:
        print '| - K =', n, '\tscore:  %.4f' % score #, '\tKweighted: %.4f' % weightedScore
        evaluate(zip(results, bids), gs)

    print 'END silhouetteAnalysis'
    
    return scores


In [5]:
def getSnippetForBusinessAndToken(business_id, token, charwinsize=40):
    index = session.execute('''
        SELECT review_id, start, "end" FROM index
        WHERE "isName" = false AND business_id = :business_id AND token = :token
        ORDER BY business_id LIMIT 1
    ''', {'business_id': business_id, 'token': token})
    
    review_id, start, end = list(index)[0]
    substring = 'substring(text, %d, %d)' % (max(0, start - charwinsize), 2*charwinsize)
    mention = session.execute('''
        SELECT %s
        FROM review WHERE review_id = :review_id
        LIMIT 1
    ''' % substring, {'review_id': review_id})
    return list(mention)[0][0].replace('\n', ' ').strip()


In [6]:
def evaluate(results, gs):
    a = []
    b = []
    
    tr = {}
    for cid, name in enumerate(list(set(gs.values()))):
        if name in set(['others', 'other']):
            continue
        tr[name] = cid
    
    for cid, bid in results:
        if gs[bid] in set(['others', 'other']):
            continue
        a.append(cid)
        b.append(tr[gs[bid]])
        
        

    #print '\t|NMI:\t\t%.4f' % normalized_mutual_info_score(a,b)
    #print '\t|AMI:\t\t%.4f' % adjusted_mutual_info_score(a, b)
    print '|\t|randIDX:\t%.4f' % adjusted_rand_score(a, b)
    
    print '|\t|clusterP:\t',
    purity = 0.0
    for cid, group in groupby(sorted(zip(a,b), key=itemgetter(0)), key=itemgetter(0)):
        labels = map(itemgetter(1), group)
        N = float(len(labels))
        n = Counter(labels).most_common()[0][1]
        print '(%d:%.0f%%),' % (N, 100*n/N),
        purity += (float(n)/N) * (float(N)/len(a))
    print '\n\t|purity:\t%.2f%%' % (100*purity)
        
            

In [7]:
def run(algorithm=lambda n_clusters: KMeans(n_clusters=n_clusters), tokens=('basil', 'cocktail', 'lemon'), repeat=1, Ks=range(2,6)):
    for token in tokens:
        print '=' * 33        
        print token
        print '=' * 33        
        business_id, vectors = getVectorsForToken(token, 10000, 'vector_backup')
        #business_id, vectors = getVectorsForToken(token, 10000)
        
        '''
        svd = TruncatedSVD(n_components=50)
        vectors = svd.fit_transform(vectors)
        
        pca = PCA(n_components=150)
        vectors = pca.fit_transform(vectors)
        '''
        
        gs = GS[token]
        scores = silhouetteAnalysis(vectors, business_id, gs, algorithm=algorithm, repeat=repeat)
        n, score, weightedResults, results = scores[0]
        results = sorted(zip(results, business_id))
        '''
        for cluster_id, payload in groupby(results, key=itemgetter(0)):
            bids = map(itemgetter(1), payload)
            print ' __________'
            print '|CLUSTER', cluster_id, '|', 'size:', len(bids) 
            print ' ----------'
            bids = random.sample(bids, min(5, len(bids)))
            for bid in bids:
                print '- ...%s...' % getSnippetForBusinessAndToken(bid, token)
        '''

        print
        print

In [8]:
run(KMeans, Ks=range(1, 3), repeat=5)

basil
silhouetteAnalysis START
| - # of restaurant vectors:  324
| Ranking of different Ks:
| - K = 2 	score:  0.0696
|	|randIDX:	0.1269
|	|clusterP:	(176:53%), (142:60%), 
	|purity:	55.97%
| - K = 3 	score:  0.0631
|	|randIDX:	0.2336
|	|clusterP:	(33:82%), (107:67%), (178:61%), 
	|purity:	65.41%
| - K = 7 	score:  0.0593
|	|randIDX:	0.2241
|	|clusterP:	(30:87%), (67:54%), (23:52%), (5:40%), (22:77%), (145:69%), (26:88%), 
	|purity:	67.92%
| - K = 4 	score:  0.0539
|	|randIDX:	0.3567
|	|clusterP:	(127:80%), (65:57%), (42:76%), (84:73%), 
	|purity:	72.96%
| - K = 6 	score:  0.0469
|	|randIDX:	0.1945
|	|clusterP:	(42:71%), (81:86%), (32:56%), (94:46%), (45:51%), (24:92%), 
	|purity:	64.78%
| - K = 5 	score:  0.0455
|	|randIDX:	0.2140
|	|clusterP:	(6:50%), (107:74%), (31:90%), (120:42%), (54:87%), 
	|purity:	65.09%
END silhouetteAnalysis


cocktail
silhouetteAnalysis START
| - # of restaurant vectors:  422
| Ranking of different Ks:
| - K = 2 	score:  0.1369
|	|randIDX:	0.5474
|	|clusterP

In [9]:
#run(AgglomerativeClustering, repeat=20)

In [10]:
from sklearn.metrics.pairwise import cosine_similarity as cosine
from scipy.spatial.distance import euclidean

def bow2vec(tokens, weights):
    tokens = filter(lambda tok: tok in model, tokens)
    vector = map(lambda token: model[token], tokens)
    vector = [w*v for w, v in zip(weights, vector)]
    vector = np.asarray(vector).sum(0)
    return vector/vector.sum()

def query(token, query, weights):
    print AsciiTable([['Query Term', 'Context Vectors'], [token, str(zip(query, weights))]]).table
    bids, vectors = getVectorsForToken(token, 10000, 'vector_backup')
    gs = GS[token]

    vector = bow2vec(query, weights)
#    cosines = [(idx, weight*cosine([vector], [Vr])[0][0]) for idx, Vr in enumerate(vectors)]
    #cosines = [(idx, -1 * weight*euclidean(vector, Vr)) for idx, Vr in enumerate(vectors)]
    #if weight < 0:
    #    cosines = [(idx, ((euclidean(vector, Vr) + euclidean(vector, basil) / euclidean(vector, Vr)))) for idx, Vr in enumerate(vectors)]
    #else:
    #    cosines = [(idx, (1.0 - (euclidean(vector, Vr) + euclidean(vector, basil) / euclidean(vector, Vr)))) for idx, Vr in enumerate(vectors)]
    
    vectors = [Vr/float(sum(Vr)) for Vr in np.asarray(vectors)]
    cosines = [(idx, cosine([vector], [Vr])[0][0]) for idx, Vr in enumerate(vectors)]
    
    #cosines = [(idx, cosine([vector], [np.asarray(Vr)/float(sum(Vr))])[0][0]) for idx, Vr in enumerate(vectors)]
    
    cosines.sort(key=itemgetter(1), reverse=True)
    rank = 0
    for idx, cos in cosines[:20]:
        rank += 1
        bid = bids[idx]
        print '%s %.2f %s\t ...%s...' % ("{0: >2}".format(rank), cos, gs[bid], getSnippetForBusinessAndToken(bid, token))
    print '-' * 88
    return [gs[bids[idx]] for idx, cos in cosines], {bids[idx]: cos for idx, cos in cosines}

URL = 'http://j.pc.cs.cmu.edu:5000/baseline_search/Pittsburgh/0.0/%s/%s'
def bm25(query, token, bid2cosine):
    print AsciiTable([['BM25 query:', query]]).table
    gs = GS[token]
    query = query.split()
    query = '%7C'.join(query)
    weight = '%7C'.join(['1',] * len(query))
    url = URL % (query, weight)

    response = urllib.urlopen(url)
    data = json.loads(response.read())
    bids = map(itemgetter('business_id'), data['business'])
    for rank, business in enumerate(data['business'][:20]):
        print str(rank+1).rjust(2, ' '),
        print ('%.2f' % bid2cosine.get(business['business_id'], 0.0)).rjust(5, '+'),
        print gs.get(business['business_id'], '???').rjust(10, ' '), '  ',
        #print business['name']
        
        mentions = data['review'][business['business_id']]
        mentions = filter(lambda mention: token in mention['text'], mentions)
        if len(mentions) == 0:
            print '(no mentions of %s)' % token
            continue
        mention = random.sample(mentions, 1)[0]['text']
        idx = mention.index(token)
        win = 35
        mention = mention[max(0, idx-win):idx+win+len(token)]
        print mention.replace('\n', ' ')
        
    print '-' * 88
        
    return [gs.get(bid, 'NotFound') for bid in bids]
    

In [11]:
def dcg(rank, k):
    out = 0.0
    for _k in range(k):
        out += ((2**rank[_k]) - 1) / log(_k+1+1, 2)
    return out
        
def ndcg(ranks, target, k=30):
    k = min(k, len(ranks))
    ranks = [rank == target and 1.0 or 0.0 for rank in ranks]
    idea_ranks = sorted(ranks[:], reverse=True)
    outs = []
    for _k in range(k):
        outs.append(dcg(ranks, _k+1)/dcg(idea_ranks, _k+1))
    return outs

def precision(ranks, target, k=30):
    k = min(k, len(ranks))
    ranks = [rank == target and 1.0 or 0.0 for rank in ranks]
    outs = []
    for _k in range(k):
        outs.append(Counter(ranks[:_k+1])[1]/float(_k+1))
    return outs

def recall(ranks, target, N, k=30):
    k = min(k, len(ranks))
    ranks = [rank == target and 1.0 or 0.0 for rank in ranks]
    outs = []
    for _k in range(k):
        outs.append(Counter(ranks[:_k+1])[1]/N)
    return outs

def plot(system, baseline, target, N):
    data = []
    systemColor = ('rgb(22, 96, 167)')
    baselineColor = ('rgb(215, 96, 92)')
    data.append(go.Scatter(
        x = range(1, len(system) + 1),
        y = ndcg(system, target),
        mode = 'lines',
        name = 'System nDCG',
        line = dict(
            color = systemColor,
            width = 3,
            dash = 'line'
        )
    ))
    data.append(go.Scatter(
        x = range(1, len(baseline) + 1),
        y = ndcg(baseline, target),
        mode = 'lines',
        name = 'Baseline nDCG',
        line = dict(
            color = baselineColor,
            width = 3,
            dash = 'line'
        )
    ))
    data.append(go.Scatter(
        x = range(1, len(system) + 1),
        y = precision(system, target),
        mode = 'lines',
        name = 'System Precision',
        line = dict(
            color = systemColor,
            width = 1,
            dash = 'dot'
        )
    ))
    data.append(go.Scatter(
        x = range(1, len(baseline) + 1),
        y = precision(baseline, target),
        mode = 'lines',
        name = 'Baseline Precision',
        line = dict(
            color = baselineColor,
            width = 1,
            dash = 'dot'
        )
    ))
    data.append(go.Scatter(
        x = range(1, len(system) + 1),
        y = recall(system, target, N),
        mode = 'lines',
        name = 'System Recall',
        line = dict(
            color = systemColor,
            width = 1,
            dash = 'dash'
        )
    ))
    data.append(go.Scatter(
        x = range(1, len(baseline) + 1),
        y = recall(baseline, target, N),
        mode = 'lines',
        name = 'Baseline Recall',
        line = dict(
            color = ('rgb(255, 146, 142)'),
            width = 1,
            dash = 'dash'
        )
    ))
    
    f1 = []
    for p, r in zip(precision(baseline, target), recall(baseline, target, N)):
        if p+r == 0:
            f1.append(0.0)
        else:
            f1.append(2*p*r/(p+r))
    data.append(go.Scatter(
        x = range(1, len(baseline) + 1),
        y = f1,
        mode = 'lines',
        name = 'Baseline F1',
        line = dict(
            color = ('rgb(252, 214, 214)'),
            width = 5,
            dash = 'line'
        )
    ))
    
    f1 = []
    for p, r in zip(precision(system, target), recall(system, target, N)):
        if p+r == 0:
            f1.append(0.0)
        else:
            f1.append(2*p*r/(p+r))
    data.append(go.Scatter(
        x = range(1, len(system) + 1),
        y = f1,
        mode = 'lines',
        name = 'System F1',
        line = dict(
            color = ('rgb(201, 222, 255)'),
            width = 5,
            dash = 'line'
        )
    ))

    return py.iplot(list(reversed(data)), filename='searchlens')

def search(token, naive, target, terms, weights):
    #baseline = query(token, [token], [1])
    system, bid2cosine = query(token, terms + [token,], weights + [1,])
    baseline = bm25(naive, token, bid2cosine)
    N = float(sum([outcome == target and 1.0 or 0.0 for outcome in GS[token].values()]))
    return plot(system, baseline, target, N)
    

In [12]:
search('cocktail', 'shrimp cocktail sauce', 'sauce', ['seafood','drinks'], [1,-1])

+------------+---------------------------------------------------+
| Query Term | Context Vectors                                   |
+------------+---------------------------------------------------+
| cocktail   | [('seafood', 1), ('drinks', -1), ('cocktail', 1)] |
+------------+---------------------------------------------------+
 1 0.32 sauce	 ...lse can you find unlimited sushi, shrimp cocktail, crawfish, clams, hot and sour...
 2 0.28 sauce	 ...i, middle neck clams, and dumplings, and cocktail shrimp. EVERYTHING was amazing...
 3 0.27 sauce	 ...ll, probably U-20 count which for Shrimp Cocktail is tiny. The wings were very d...
 4 0.25 sauce	 ...- small salad bar, seafood salad, shrimp cocktail, etc. were good except a few o...
 5 0.24 sauce	 ...I also had the shrimp appetizer (5 giant cocktail shrimp).    Took a long time t...
 6 0.23 drinks	 ...plate of coconut shrimp. We also ordered cocktails: the Geisha's Delight and a Z...
 7 0.22 drinks	 ...nting  and uninspired meal. We or

In [13]:
search('lemon', 'lemon drinks', 'drinks', ['cocktail', 'drinks', 'desserts'], [1, 1,-1])

+------------+------------------------------------------------------------------+
| Query Term | Context Vectors                                                  |
+------------+------------------------------------------------------------------+
| lemon      | [('cocktail', 1), ('drinks', 1), ('desserts', -1), ('lemon', 1)] |
+------------+------------------------------------------------------------------+
 1 0.59 drinks	 ...amble cocktail (gin, blackberry cordial, lemon) and the hallumi. The cocktail wa...
 2 0.49 drinks	 ...was unbelievable - who knew rosemary and lemon flavors make a good cocktail!?! A...
 3 0.46 drinks	 ...arm pita bread. It had a really distinct lemon flavor that I loved. I was surpri...
 4 0.45 dishes	 ...peno essence. The rye whiskey drink with lemon was one of the best cocktails I'v...
 5 0.45 drinks	 ...was just a sour glass of Ginger Ale and Lemon. The Deuce's Wild Mint Lemonade w...
 6 0.45 drinks	 ...o Bloody Mary mix, vodka, old bay rim, a lemon and a lime

In [14]:
search('lemon', 'lemon desserts', 'dessert', ['desserts'], [1])

+------------+---------------------------------+
| Query Term | Context Vectors                 |
+------------+---------------------------------+
| lemon      | [('desserts', 1), ('lemon', 1)] |
+------------+---------------------------------+
 1 0.57 dessert	 ...ing and so creative (bruleed grapefruit, lemon mousse with meringue and shortbre...
 2 0.55 dishes	 ...re a bit underdone and had been hit with lemon juice or some other acid before s...
 3 0.51 dessert	 ...e, and totally loved it! Why? 2 reasons: LEMON GINGER COOKIES and BACON MAC n CH...
 4 0.50 dessert	 ...something I wanted to finish. My fish -- lemon sage ahi tuna -- was decent. My k...
 5 0.48 dessert	 ...t's so intoxicatingly delicious. And the lemon lavender cupcakes...that's an ent...
 6 0.46 dessert	 ...brûlée was decent, but a little too much lemon flavor. You can see the granules...
 7 0.44 dishes	 ...sert, I ordered the yogurt, he the Meyer lemon. The Meyer lemon with sesame cake...
 8 0.44 dishes	 ...ssive array

In [15]:
search('lemon', 'lemon chicken', 'dishes', ['chicken'], [1])

+------------+--------------------------------+
| Query Term | Context Vectors                |
+------------+--------------------------------+
| lemon      | [('chicken', 1), ('lemon', 1)] |
+------------+--------------------------------+
 1 0.72 dishes	 ...the special and told me I should try the lemon chicken. His name was Lou and he...
 2 0.62 dishes	 ...re a bit underdone and had been hit with lemon juice or some other acid before s...
 3 0.60 dessert	 ...something I wanted to finish. My fish -- lemon sage ahi tuna -- was decent. My k...
 4 0.58 dishes	 ...red soups, salads, sushi, fried rice and lemon chicken! All were great! When you...
 5 0.56 dishes	 ...the orange chicken, sesame chicken, and lemon chicken. All are delicious. I've...
 6 0.54 dishes	 ...p. It's almost in edible. It tasted like lemon. Period. Not only that they clear...
 7 0.54 dishes	 ...as, and cracked wheat) and Shish Tawook (lemon mint chicken) and both were out o...
 8 0.53 dishes	 ...that they don't even g

In [16]:
search('basil', 'basil cocktail', 'drinks', ['pizza', 'cocktail'], [-1,1])

+------------+------------------------------------------------+
| Query Term | Context Vectors                                |
+------------+------------------------------------------------+
| basil      | [('pizza', -1), ('cocktail', 1), ('basil', 1)] |
+------------+------------------------------------------------+
 1 0.54 drinks	 ...eason) once had one with Braggs and Thai basil at Cantina. The fact that my frie...
 2 0.53 drinks	 ...Complete with your strawberry slices and basil, this thing is excessively easy t...
 3 0.45 drinks	 ...$12) It had Absolut Vodka, Strawberries, Basil and Lemonade and my wife had a "H...
 4 0.45 redsauce	 ...atering thinking about it right now....  Basil Gin-tini: Fresh muddled basil, fr...
 5 0.39 drinks	 ...menu is also delightful! The grapefruit basil margarita calls my name regularly...
 6 0.38 drinks	 ...r friendly items.  I had the mozzarella, basil, tomato salad (let's just call it...
 7 0.38 drinks	 ...ab and not much filler. I had the tomato b

In [17]:
search('basil', 'basil thai', 'hotdish', ['thai','food'], [1,1])

+------------+------------------------------------------+
| Query Term | Context Vectors                          |
+------------+------------------------------------------+
| basil      | [('thai', 1), ('food', 1), ('basil', 1)] |
+------------+------------------------------------------+
 1 0.53 hotdish	 ...te rice wasn't enough. The chicken spicy basil fried rice was flavorful. The Tha...
 2 0.52 hotdish	 ...a dry sandwich with cheese, chicken and basil. next time I'm going over to Prim...
 3 0.49 other	 ...d limes individually, and they have thai basil which you can't find any where el...
 4 0.49 hotdish	 ...auce, walnut shrimp, Mongolian beef, and basil chicken.  Hot and sour soup is be...
 5 0.46 other	 ...ose rice, soup or salad. I got the spicy basil duck with rice and the beef pad s...
 6 0.45 hotdish	 ...isits has proved to be very bad.  We got basil fried rice and the rice was sever...
 7 0.45 hotdish	 ...general top 10) on the planet. The Spicy Basil Noodles are to die for. 

In [18]:
search('basil', 'thai basil curry', 'hotdish', ['curry', 'thai'], [1,1])

+------------+-------------------------------------------+
| Query Term | Context Vectors                           |
+------------+-------------------------------------------+
| basil      | [('curry', 1), ('thai', 1), ('basil', 1)] |
+------------+-------------------------------------------+
 1 0.67 hotdish	 ...hai curry chicken, sushi rolls, and Thai basil curry. We really liked everything...
 2 0.58 hotdish	 ...te rice wasn't enough. The chicken spicy basil fried rice was flavorful. The Tha...
 3 0.56 hotdish	 ...ous. The spicy noodles had lots of fresh basil cooked into the spicy noodles and...
 4 0.54 hotdish	 ...reen beans, broccoli, eggplant, and Thai basil. I took a few sips of the broth a...
 5 0.53 hotdish	 ...e dishes.  The stir fries -i.e. eggplant basil, basil leaves- are both awesome a...
 6 0.52 hotdish	 ...nner in a tropical garden. I ordered the Basil Eggplant, which was delicious, no...
 7 0.50 hotdish	 ...y (Roasted Duck, Tomatoes, Bell Peppers, Basil Leaves and Pin

In [19]:
search('lemon', 'lemon chicken beef noodles', 'dishes', ['chicken', 'beef', 'noodles'], [1,1,1])

+------------+-------------------------------------------------------------+
| Query Term | Context Vectors                                             |
+------------+-------------------------------------------------------------+
| lemon      | [('chicken', 1), ('beef', 1), ('noodles', 1), ('lemon', 1)] |
+------------+-------------------------------------------------------------+
 1 0.59 dishes	 ...the special and told me I should try the lemon chicken. His name was Lou and he...
 2 0.56 dessert	 ...something I wanted to finish. My fish -- lemon sage ahi tuna -- was decent. My k...
 3 0.54 dishes	 ...re a bit underdone and had been hit with lemon juice or some other acid before s...
 4 0.54 dishes	 ...red soups, salads, sushi, fried rice and lemon chicken! All were great! When you...
 5 0.53 dishes	 ...the orange chicken, sesame chicken, and lemon chicken. All are delicious. I've...
 6 0.51 dishes	 ...p. It's almost in edible. It tasted like lemon. Period. Not only that they clear...

In [20]:
search('lemon', 'lemon grass thai', 'grass', ['grass', 'thai', 'noodles', 'spicy', 'basil', 'curry', 'duck'], [1,1,1,1,1,1,1])


+------------+------------------------------------------------------------------------------------------------------------------+
| Query Term | Context Vectors                                                                                                  |
+------------+------------------------------------------------------------------------------------------------------------------+
| lemon      | [('grass', 1), ('thai', 1), ('noodles', 1), ('spicy', 1), ('basil', 1), ('curry', 1), ('duck', 1), ('lemon', 1)] |
+------------+------------------------------------------------------------------------------------------------------------------+
 1 0.64 dishes	 ...s occasionally. I *highly* recommend the Lemon Grass Beef. That is the most amaz...
 2 0.62 others	 ...Thai food so I thought we would give the Lemon Grass Cafe a try. We arrived a li...
 3 0.60 dishes	 ...ely it was made to order. I ordered Thai Lemon Beef on noodles. As soon as I saw...
 4 0.60 grass	 ...in Pittsburgh. Spring r